# Set up

- Download / Install libraries needed in the workspace


## Load modules

In [1]:
# Use the os package to interact with the environment
import os

# Use StringIO for working with file contents
from io import StringIO

# Bring in Pandas for Dataframe functionality
import pandas as pd

# numpy for basics
import numpy as np

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Import seaborn for plots
import seaborn as sns

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

/tmp/ipykernel_185/4207620771.py:24: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Load functions

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}')
    !$command

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))
    
# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Define bash and python environmental variables / WD

##### bash

In [3]:
%%bash

mkdir -p /home/jupyter/longwas_workdir
mkdir -p  /home/jupyter/longwas_workdir/corriellnetpd_geno
mkdir -p /home/jupyter/longwas_workdir/clinical_qc

mkdir -p /home/jupyter/longwas_workdir
mkdir -p  /home/jupyter/longwas_workdir/ppmi_geno

mkdir -p /home/jupyter/longwas_workdir
mkdir -p  /home/jupyter/longwas_workdir/pdgen_geno


WORK_DIR=/home/jupyter/longwas_workdir
cd $WORK_DIR

##### python

In [5]:
WORK_DIR = "/home/jupyter/longwas_workdir/"

## Get some environmental variables with GCP project metadata and paths to buckets

In [6]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
#WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

print(BILLING_PROJECT_ID)
print(WORKSPACE_NAMESPACE)
print(WORKSPACE_NAME)
print(WORKSPACE_BUCKET)


## GP2 v3.0
GP2_RELEASE_PATH = 'gs://gp2tier2/release3_31102022'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
GP2_WGS_PATH = f'{GP2_RELEASE_PATH}/wgs/plink/bfiles'
link_to_cloud_console_gcs("Access GP2 bucket: ", "HERE", GP2_RELEASE_PATH)

## NEXTFLOW WS BUCKET
NEXTFLOW_WD_BUCKET = 'gs://long-gwas'

link_to_cloud_console_gcs("Take a look at the Nextflow WD bucket: ", "HERE!", NEXTFLOW_WD_BUCKET)

link_to_cloud_console_gcs("Take a look at the workspace bucket: ", "HERE!", WORKSPACE_BUCKET)

terra-05d0f4c6
GP2-Bioinformatics-Courses
MEXICO_WORKSHOP
gs://fc-secure-cbdcecad-15a2-473b-8df9-841de062728d


### Get plink binaries

In [6]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink is already installed in /home/jupyter/tools/"

else
echo -e "Dowloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink is already installed in /home/jupyter/tools/"

else
echo -e "Dowloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

cd ~

Plink is already installed in /home/jupyter/tools/
Plink is already installed in /home/jupyter/tools/


# Load the data

### Get access to the clinical data

In [ ]:
netpd = gcs_read_csv(f'{WORKSPACE_BUCKET}/netpdls1/NET_PD_LS1_20230627_processed_unreviewed.csv')
netpdcorriell_ids = gcs_read_csv(f'{WORKSPACE_BUCKET}/netpdls1/IDmatch.csv')

print(netpdcorriell_ids)

netpdcorriell = netpdcorriell_ids.merge(netpd, on = 'participant_id')
netpdcorriell.info()
netpdcorriell.head()

### Get access tho imputed data


- Copy data on workspace
- Filter according to the 


In [15]:
get_names = shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -R {GP2_IMPUTED_GENO_PATH}/EUR/chr7* {WORK_DIR}')

Executing: gsutil -u terra-05d0f4c6 -m cp -R gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr7* /home/jupyter/longwas_workdir/
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr7_EUR_release3.pgen...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr7_EUR_release3.psam...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr7_EUR_release3.log...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr7_EUR_release3.pvar...
- [4/4 files][  5.9 GiB/  5.9 GiB] 100% Done  57.4 MiB/s ETA 00:00:00           
Operation completed over 4 objects/5.9 GiB.                                      


### Filter Corriell IDs and then convert to VCF files as this is the standard inpiut the tool uses

#### Get the IDs

In [ ]:
corriell_keep = list(netpdcorriell_ids['GP2sampleID'])
corriell_keep

with open(f'{WORK_DIR}corriell_ids.txt', 'w') as file:
    for myid in corriell_keep:
        # write each item on a new line
        file.write("%s\n" % myid)
    print('Done')


In [6]:
%%bash -s "$WORK_DIR"
cd $1
tail corriell_ids.txt

CORIELL_003785_s1
CORIELL_004157_s1
CORIELL_003667_s1
CORIELL_003501_s1
CORIELL_003877_s1
CORIELL_003260_s1
CORIELL_003446_s1
CORIELL_004118_s1
CORIELL_003419_s1
CORIELL_003514_s1


#### Perform the filtering

In [64]:
%%bash -s "$WORK_DIR"
cd $1
corriellnetpd_geno
for chnum in {1..22};
do
/home/jupyter/tools/plink2 \
    --pfile EUR/chr"$chnum"_EUR_release3 \
    --keep corriell_ids.txt \
    --make-pgen \
    --out corriellnetpd_geno/corriell_chr"$chnum"_EUR_release3
done

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to corriellnetpd_geno/corriell_chr13_EUR_release3.log.
Options in effect:
  --keep corriell_ids.txt
  --make-pgen
  --out corriellnetpd_geno/corriell_chr13_EUR_release3
  --pfile EUR/chr13_EUR_release3

Start time: Sun Jul  2 18:18:38 2023
60283 MiB RAM detected; reserving 30141 MiB for main workspace.
Using up to 16 threads (change this with --threads).
11002 samples (4271 females, 6731 males; 11002 founders) loaded from
EUR/chr13_EUR_release3.psam.
711517 variants loaded from EUR/chr13_EUR_release3.pvar.
1 binary phenotype loaded (6852 cases, 3875 controls).
--keep: 422 samples remaining.
422 samples (156 females, 266 males; 422 founders) remaining after main
filters.
422 cases and 0 controls remaining after main filters.
Writing corriellnetpd_geno/corriell_chr13_EUR_release3.psam ... done.
Writing corriellnetpd_geno/cor

bash: line 2: corriellnetpd_geno: command not found


#### Creat VCFs

In [91]:
%%bash -s "$WORK_DIR"
cd $1

for chnum in {1..22};
do
/home/jupyter/tools/plink2 \
    --pfile corriellnetpd_geno/corriell_chr"$chnum"_EUR_release3 \
    --recode vcf \
    --out corriellnetpd_geno/corriell_chr"$chnum"_EUR_release3
done

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to corriellnetpd_geno/corriell_chr1_EUR_release3.log.
Options in effect:
  --export vcf
  --out corriellnetpd_geno/corriell_chr1_EUR_release3
  --pfile corriellnetpd_geno/corriell_chr1_EUR_release3

Start time: Sun Jul  2 18:46:00 2023
60283 MiB RAM detected; reserving 30141 MiB for main workspace.
Using up to 16 threads (change this with --threads).
422 samples (156 females, 266 males; 422 founders) loaded from
corriellnetpd_geno/corriell_chr1_EUR_release3.psam.
1567941 variants loaded from
corriellnetpd_geno/corriell_chr1_EUR_release3.pvar.
1 binary phenotype loaded (422 cases, 0 controls).
--export vcf to corriellnetpd_geno/corriell_chr1_EUR_release3.vcf ... 0%0%1%1%2%2%3%3%4%4%5%5%6%6%7%7%8%8%9%9%10%10%11%11%12%12%13%13%14%14%15%15%16%16%

# Push files to the WD bucket

#### Copy to your path

In [6]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORK_DIR}corriellnetpd_geno/* {NEXTFLOW_WD_BUCKET}/long-gwas/DATA/amc/netpdcorriell/geno/')

Executing: gsutil -u terra-05d0f4c6 -m cp /home/jupyter/longwas_workdir/corriellnetpd_geno/* gs://long-gwas/long-gwas/DATA/amc/netpdcorriell/geno/
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr10_EUR_release3.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr10_EUR_release3.pgen [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr10_EUR_release3.psam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr10_EUR_release3.pvar [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do 

Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr20_EUR_release3.psam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr20_EUR_release3.pvar [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr21_EUR_release3.psam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr21_EUR_release3.pgen [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr20_EUR_release3.vcf [Content-Type=text/vcard]...
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr2_EUR_release3.pgen [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/longwas_workdir/corriellnetpd_geno/corriell_chr21_EUR_release3.pvar [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/longwas

# Process clinical data

We need to generate two files
- Phenotype file

- Covariates file

### Filter out patients missing genotype data

In [ ]:
sam = pd.read_csv(f'{WORK_DIR}corriellnetpd_geno/corriell_chr10_EUR_release3.psam', sep = "\t")
print(sam.shape)

In [ ]:
genoids = sam.iloc[:,0].to_list()
netpd_qc = netpdcorriell[netpdcorriell['GP2sampleID'].isin(genoids)].reset_index(drop=True).copy()

In [ ]:
netpd_qc_nona = netpd_qc[~netpd_qc['updrs_part_iii_summary_score'].isna()]
netpd_qc_nona = netpd_qc_nona.rename(columns={"GP2sampleID": "IID"})
netpd_qc_nona.head()

### Get phenofile

In [60]:
pheno = netpd_qc_nona[['IID', 'visit_name', 
                       'updrs_part_ii_summary_score', 'updrs_part_iii_summary_score']]

print('Number UPDRS II NAs: ', pheno['updrs_part_ii_summary_score'].isna().sum().sum())
print('Number UPDRS III NAs: ', pheno['updrs_part_iii_summary_score'].isna().sum().sum())

#pheno.head()
pheno.to_csv(f'{WORK_DIR}clinical_qc/updrspheno_netpdCorriell.tsv', sep="\t")

Number UPDRS II NAs:  0
Number UPDRS III NAs:  0


### Get covars file

In [63]:
covars = netpd_qc_nona[['IID', 'age_at_baseline', 'sex']]\
                      .drop_duplicates(subset='IID', keep="first")
covars.to_csv(f'{WORK_DIR}clinical_qc/covars_netpdCorriell.tsv', sep="\t")

### Copy data to the bucket

In [79]:
#os.listdir(f'{WORK_DIR}clinical_qc')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORK_DIR}/clinical_qc/* {NEXTFLOW_WD_BUCKET}/long-gwas/DATA/amc/netpdcorriell/clinical/')

Executing: gsutil -u terra-05d0f4c6 -m cp /home/jupyter/longwas_workdir//clinical_qc/* gs://long-gwas/long-gwas/DATA/amc/netpdcorriell/clinical/
Copying file:///home/jupyter/longwas_workdir//clinical_qc/updrspheno_netpdCorriell.tsv [Content-Type=text/tab-separated-values]...
Copying file:///home/jupyter/longwas_workdir//clinical_qc/covars_netpdCorriell.tsv [Content-Type=text/tab-separated-values]...
/ [2/2 files][106.1 KiB/106.1 KiB] 100% Done                                    
Operation completed over 2 objects/106.1 KiB.                                    
